In [1]:
import pandas as pd
from datetime import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib
import time
import numpy as np
%matplotlib inline
%load_ext tensorboard

### data preprocessing